<a href="https://colab.research.google.com/github/pias97/Image-Colorization-Using-AutoEncoder/blob/master/Image_Colorization_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Neccessary Library

In [ ]:
from keras.layers import Conv2D, UpSampling2D, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf
import keras
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
!pip install tensorflow-gpu

#Importing VGG16 and taking only the feature extraction part

In [ ]:
from keras.applications.vgg16 import VGG16
vggmodel = VGG16()
newmodel = Sequential() 
#num = 0
for i, layer in enumerate(vggmodel.layers):
    if i<19:          #Only up to 19th layer to include feature extraction only
      newmodel.add(layer)

553476096/553467096 [==============================] - 3s 0us/step


In [ ]:
# newmodel.summary()

In [ ]:
#We don't want to train these layers again, so False.
for layer in newmodel.layers:
  layer.trainable=False    

In [ ]:
path = "/content/drive/MyDrive/Work/Quantum AI Course/Assignments/CNN/Dataset/small/train"
test_path = "/content/drive/MyDrive/Work/Quantum AI Course/Assignments/CNN/Dataset/small/test/testing b_w"

In [ ]:
#Normalize images - divide by 255
train_datagen = ImageDataGenerator(rescale=1. / 255)

train = train_datagen.flow_from_directory(path, target_size=(224, 224), batch_size=32, class_mode=None)
test = train_datagen.flow_from_directory(test_path, target_size=(224, 224), batch_size=32, class_mode=None)

Found 3000 images belonging to 1 classes.
Found 20 images belonging to 1 classes.


#Converting RGB image to lab image for feature extraction

In [ ]:
X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0]) 
      Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128, 
      #so we divide the values by 128 to restrict values to between -1 and 1.
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)

(32, 224, 224, 1)
(32, 224, 224, 2)


In [ ]:
#Early Stopping
Early_Stopper = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=10,mode="max")
Checkpoint_Model = tf.keras.callbacks.ModelCheckpoint(monitor="loss",
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      filepath="./modelcheck")

In [ ]:
#Encoder
vggfeatures = []
for i, sample in enumerate(X):
  sample = gray2rgb(sample)
  sample = sample.reshape((1,224,224,3))
  prediction = newmodel.predict(sample)
  prediction = prediction.reshape((7,7,512)) 
  vggfeatures.append(prediction)
vggfeatures = np.array(vggfeatures)
print(vggfeatures.shape)

(32, 7, 7, 512)


In [ ]:
#Decoder
model = Sequential()

model.add(Conv2D(256, (3,3), activation='relu', padding='same', input_shape=(7,7,512)))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
# keras.layers.Dropout(0.25)
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
# keras.layers.Dropout(0.25)
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
# keras.layers.Dropout(0.5)
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
# model.summary()

In [ ]:
#Model Compile
model.compile(optimizer='Adam', loss='mse' , metrics=['accuracy'])

In [ ]:
model.fit(vggfeatures, Y, verbose=1, epochs=600, batch_size=128)

Epoch 1/600
1/1 [==============================] - 1s 1s/step - loss: 0.3078 - accuracy: 0.5378
Epoch 2/600
1/1 [==============================] - 1s 973ms/step - loss: 0.5732 - accuracy: 0.3445
Epoch 3/600
1/1 [==============================] - 1s 961ms/step - loss: 0.6150 - accuracy: 0.3613
Epoch 4/600
1/1 [==============================] - 1s 961ms/step - loss: 0.5783 - accuracy: 0.6686
Epoch 5/600
1/1 [==============================] - 1s 956ms/step - loss: 0.4500 - accuracy: 0.6778
Epoch 6/600
1/1 [==============================] - 1s 944ms/step - loss: 0.6180 - accuracy: 0.6778
Epoch 7/600
1/1 [==============================] - 1s 954ms/step - loss: 0.4778 - accuracy: 0.6802
Epoch 8/600
1/1 [==============================] - 1s 954ms/step - loss: 0.4614 - accuracy: 0.6802
Epoch 9/600
1/1 [==============================] - 1s 968ms/step - loss: 0.4632 - accuracy: 0.6794
Epoch 10/600
1/1 [==============================] - 1s 940ms/step - loss: 0.3981 - accuracy: 0.6784
Epoch 11/600

#Model Saving

In [ ]:
# saving the model
from tensorflow.keras.models import load_model
model.save('/content/drive/MyDrive/Work/Quantum AI Course/Assignments/CNN/Dataset/small/colorization_epoch-600_acc-81.h5')

#Prdiction using model

In [ ]:
# loading the model
model = tf.keras.models.load_model('/content/drive/MyDrive/Work/Quantum AI Course/Assignments/CNN/Dataset/small/colorization_epoch-600_acc-81.h5',
                                   custom_objects=None,
                                   compile=True)

In [ ]:
test_path = '/content/model_train/0aae87f4fa55cc79.jpg'
i = 0
# for idx, image in enumerate(test_path):
test = img_to_array(load_img(test_path))
test = resize(test, (224,224), anti_aliasing=True)
test*= 1.0/255
lab = rgb2lab(test)
l = lab[:,:,0]
L = gray2rgb(l)
L = L.reshape((1,224,224,3))
vggpred = newmodel.predict(L)
ab = model.predict(vggpred)
ab = ab*128
cur = np.zeros((224, 224, 3))
cur[:,:,0] = l
cur[:,:,1:] = ab
imsave('/content/model_result/result'+ 'vgg' +".jpg", lab2rgb(cur))
